In [5]:
import pandas as pd
import numpy as np

In [6]:
accel00001 = pd.read_csv('/SPHERE-Challenge/data/raw_data/public_data/train/00001/acceleration.csv')
target00001 = pd.read_csv('/SPHERE-Challenge/data/raw_data/public_data/train/00001/targets.csv')

In [7]:
def window_feat(t_start, t_end, df, columns):
    
    ''' Extract statistical features for a given window and sensor data frame df '''
    
    window_size = t_end - t_start
    window_data = df[(df['t'] >= t_start) & (df['t'] < t_end)]
    window_data = window_data[columns]
    
    feat = [] # empty features list
    fnames = [] # empty feature names list

    # quantiles
    feat = feat + window_data.quantile(q = [0, 0.25, 0.50, 0.75, 1]).values.flatten().tolist()
    fnames = ( fnames + 
              [ (column + '_W' + str(window_size) + '_0Q') for column in window_data.columns] + 
              [ (column + '_W' + str(window_size) + '_25Q') for column in window_data.columns] +
              [ (column + '_W' + str(window_size) + '_50Q') for column in window_data.columns] +
              [ (column + '_W' + str(window_size) + '_75Q') for column in window_data.columns] +
              [ (column + '_W' + str(window_size) + '_100Q') for column in window_data.columns] )
    
    # mean
    feat = feat + window_data.mean().tolist()
    fnames = fnames + [ (column + '_W' + str(window_size) + '_mean') for column in window_data.columns]
    
    # std
    feat = feat + window_data.std().tolist()
    fnames = fnames + [ (column + '_W' + str(window_size) + '_std') for column in window_data.columns]
    
    return feat, fnames

In [8]:
feats = []

for idx in target00001.index: # [0,1823]
    
    s = target00001.loc[idx,'start']
    e = target00001.loc[idx,'end']
    
    # extract features
    feat_W1, fnames_W1 = window_feat(t_start = e - 1, t_end = e, df = accel00001, columns = ['x', 'y', 'z'])
    feat_W2, fnames_W2 = window_feat(t_start = e - 2, t_end = e, df = accel00001, columns = ['x', 'y', 'z'])
    feat_W3, fnames_W3 = window_feat(t_start = e - 3, t_end = e, df = accel00001, columns = ['x', 'y', 'z'])
    feat_W4, fnames_W4 = window_feat(t_start = e - 4, t_end = e, df = accel00001, columns = ['x', 'y', 'z'])
    
    # consolidate features
    feat = feat_W1 + feat_W2 + feat_W3 + feat_W4
    fnames = fnames_W1 + fnames_W2 + fnames_W3 + fnames_W4
    
    feats.append(feat)
    
feats = pd.DataFrame(feats, columns = fnames)

In [33]:
# create features for all train data
targets = target00001.iloc[:,2:22]

In [48]:
feats = feats[np.isfinite(targets.sum(axis=1, skipna=False))]
targets = targets[np.isfinite(targets.sum(axis=1, skipna=False))]

In [52]:
import xgboost

In [54]:
xgboost.DMatrix(data = feats, label = targets)

ValueError: DataFrame for label cannot have multiple columns